<a href="https://colab.research.google.com/github/yjh111172/medical-data-analysis/blob/main/12_generate_molecules_with_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DeepChem 설치시 Keras 종속성 버전 일치를 위해 아래 환경 변수 설정 후 import 진행
import os
os.environ["TF_USE_LEGACY_KERAS"] = '1'

In [ ]:
!pip install deepchem[tensorflow]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 28.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
!pip install deepchem[pytorch]

In [ ]:
!pip install rdkit

In [ ]:
#!pip install tf_keras

In [ ]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:26] DEPRECATION WARNING: please use MorganGenerator
[19:01:27] DEPRECATION WARNING: please use MorganGenerator
[19:01:27] DEPRECATION WARNING: please use MorganGenerator
[19:01:27] DEPRECATION WARNING: please use MorganGenerator
[19:01:27] DEPRECATION WARNING: please use MorganGenerator
[19:01:27] DEPRECATI

In [ ]:
train_dataset.X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train_smiles

array(['NC(=O)NC(Cc1ccccc1)C(=O)O', 'Nc1ccc(C(=O)O)c(O)c1',
       'C=CCNC(=S)Nc1ccc(Br)cc1F', ...,
       'CC(=O)Nc1ccc(NC(=O)Cc2ccc(S(=O)(=O)N3CCCCC3)s2)cc1',
       'COc1ccc(N(C(=O)c2cccnc2)S(=O)(=O)c2cccs2)cc1',
       'COc1ccc(S(=O)(=O)N2CCCN(CCC(=O)Nc3cccc(C(F)(F)F)c3)CC2)cc1'],
      dtype=object)

In [ ]:
tokens = set()
for s in train_smiles:
  tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

In [ ]:
print(tokens)

['#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '6', '=', 'B', 'C', 'F', 'H', 'N', 'O', 'S', '[', '\\', ']', 'c', 'l', 'n', 'o', 'r', 's']


In [ ]:
print(max_length)

82


In [ ]:
from deepchem.models.optimizers import ExponentialDecay
from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder

batch_size = 100
batches_per_epoch = len(train_smiles)/batch_size
learning_rate = ExponentialDecay(0.001, 0.95, batches_per_epoch)
model = AspuruGuzikAutoEncoder(tokens, max_length, model_dir='vae', batch_size=batch_size, learning_rate=learning_rate)

In [ ]:
def generate_sequences(epochs):
  for i in range(epochs):
    for s in train_smiles:
      yield (s, s)

model.fit_sequences(generate_sequences(50))

In [ ]:
import numpy as np
from rdkit import Chem

predictions = model.predict_from_embeddings(np.random.normal(size=(1000,196)))
molecules = []

for p in predictions:
  smiles = ''.join(p)
  if Chem.MolFromSmiles(smiles) is not None:
    molecules.append(smiles) # SMILES 문자열이 유효한 분자 구조인지 확인

print()
print('Generated molecules:')

for m in molecules:
  print(m)


Generated molecules:


[20:04:53] SMILES Parse Error: extra open parentheses while parsing: CCC(ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
[20:04:53] SMILES Parse Error: check for mistakes around position 4:
[20:04:53] CCC(ccccccccccccccccccccccccccccccccccccc
[20:04:53] ~~~^
[20:04:53] SMILES Parse Error: Failed parsing SMILES 'CCC(ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc' for input: 'CCC(ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc'
[20:04:53] SMILES Parse Error: extra open parentheses while parsing: C=C(ccccccccccccccccccccccccccccccc1ccccc1
[20:04:53] SMILES Parse Error: check for mistakes around position 4:
[20:04:53] C=C(ccccccccccccccccccccccccccccccc1ccccc
[20:04:53] ~~~^
[20:04:53] SMILES Parse Error: Failed parsing SMILES 'C=C(ccccccccccccccccccccccccccccccc1ccccc1' for input: 'C=C(ccccccccccccccccccccccccccccccc1ccccc1'
[20:04:53] SMILES Parse Error: unclosed ring for input: 'C=C(cccccccccccccc